https://python.langchain.com/docs/modules/agents/how_to/custom_agent/

In [ ]:
from getpass import getpass

In [ ]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

In [ ]:
from ase.atoms import Atoms
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.constraints import UnitCellFilter
from ase.eos import calculate_eos, plot
from ase.data import reference_states, atomic_numbers
from ase.optimize import LBFGS
from ase.units import kJ

In [ ]:
from mace.calculators import mace_mp

In [ ]:
from langchain.agents import tool

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from langchain_core.pydantic_v1 import BaseModel

In [ ]:
import numpy as np

In [ ]:
from typing import List

In [ ]:
class AtomsDict(BaseModel):
    numbers: List[int]
    positions: List[List[float]]
    cell: List[List[float]]
    pbc: List[bool]

In [ ]:
AtomsDict.schema()

In [ ]:
def get_calculator(calculator_str: str = "emt"):
    if calculator_str == "emt":
        return EMT()
    elif calculator_str == "mace":
        return mace_mp(
            model="medium",
            dispersion=False,
            default_dtype="float32",
            device='cpu'
        )

In [ ]:
@tool 
def get_equilibirum_lattice(chemical_symbol: str, calculator_str: str) -> AtomsDict:
    """Returns equilibrium atoms dictionary for a given chemical symbol and a selected model specified by the calculator string"""
    atoms = bulk(name=chemical_symbol)
    atoms.calc = get_calculator(calculator_str=calculator_str)
    ase_optimizer_obj = LBFGS(UnitCellFilter(atoms))
    ase_optimizer_obj.run(fmax=0.000001)
    return AtomsDict(**{k: v.tolist() for k, v in atoms.todict().items()})

In [ ]:
@tool
def plot_equation_of_state(atom_dict: AtomsDict, calculator_str: str) -> str:
    """Returns plot of equation of state of chemical symbol for a given atoms dictionary and a selected model specified by the calculator string"""
    atoms = Atoms(**atom_dict.dict())
    atoms.calc = get_calculator(calculator_str=calculator_str)
    eos = calculate_eos(atoms)
    plotdata = eos.getplotdata()
    print(*plotdata)
    return plot(*plotdata)

In [ ]:
el = "Al"
structure_dict = get_equilibirum_lattice.invoke({"chemical_symbol": el, "calculator_str": "emt"})
plot_equation_of_state.invoke({"atom_dict": structure_dict, "calculator_str": "emt"})

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
tools = [get_equilibirum_lattice, plot_equation_of_state]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",
            # "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
            "You are very powerful assistant, but don't know current events. To calculate with emt use the calculator string emt and to calculate with mace use the calculator string mace. For each query vailidate that it contains a chemical element and a calculator string and otherwise use Alumninum as the default chemical element and emt as the default calculator string.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
lst = list(agent_executor.stream({"input": "Plot the equation of state of gold"}))  # Yeah this worked !!

In [ ]:
lst[0]

In [ ]:
lst[1]

In [ ]:
lst[2]

In [ ]:
lst[3]

In [ ]:
lst[4]

In [ ]:
lst = list(agent_executor.stream({"input": "Plot the equation of state of gold calculated with the mace model"}))  # Yeah this worked !!

In [ ]:
lst = list(agent_executor.stream({"input": "Plot the equation of state"}))  # This worked before but does not work now !!

# Lessons learned: 
* Handling types between python functions is still ugly, it is not possible to transfer ASE atoms objects. Rather we use pydantic classes, still even for those tuples are not allowed. 